# FMP API Quick Check

这个 Notebook 用来快速验证本地环境是否能从 Financial Modeling Prep (FMP) 拉取财报数据。

## 使用说明

1. 在项目根目录的 `.env` 或系统环境变量里写入 `FMP_API_KEY=<你的密钥>`。
2. 如需指定其他 `.env`，修改下方 `ENV_FILE`。
3. 运行后面的代码单元即可看到接口返回的数据。

In [3]:
import os
from datetime import date, timedelta
from pathlib import Path
from typing import Iterable
import requests

ENV_FILE = Path("../.env")  # 如需指定其他路径，修改这里

def load_env_file(path: Path) -> None:
    if not path.exists():
        return
    for raw in path.read_text(encoding="utf-8").splitlines():
        line = raw.strip()
        if not line or line.startswith("#") or "=" not in line:
            continue
        key, value = line.split("=", 1)
        os.environ.setdefault(key.strip(), value.strip().strip('"').strip("'"))

load_env_file(ENV_FILE)
FMP_API_KEY = os.getenv("FMP_API_KEY")
if not FMP_API_KEY:
    raise RuntimeError("FMP_API_KEY 未设置，请先在 .env 或环境变量中配置。")

In [4]:
# 根据需要调整要查询的股票列表与时间范围
symbols = ["AAPL", "MSFT", "NVDA"]
lookahead_days = 60  # 查询未来多少天

start_date = date.today()
end_date = start_date + timedelta(days=lookahead_days)

print(f"查询区间: {start_date} ~ {end_date}")

查询区间: 2025-11-01 ~ 2025-12-31


In [7]:
def fetch_fmp_calendar(symbols: Iterable[str], start: date, end: date) -> list[dict]:
    since = start.strftime("%Y-%m-%d")
    until = end.strftime("%Y-%m-%d")
    url = (
        "https://financialmodelingprep.com/stable/earnings-calendar?"
        f"?from={since}&to={until}&apikey={FMP_API_KEY}"
    )
    response = requests.get(url, timeout=20)
    response.raise_for_status()
    data = response.json() or []
    wanted = {symbol.upper() for symbol in symbols}
    return [item for item in data if (item.get("symbol") or "").upper() in wanted]

events = fetch_fmp_calendar(symbols, start_date, end_date)
print(f"返回 {len(events)} 条记录")
events[:5]

返回 4 条记录


[{'symbol': 'NVDA',
  'date': '2025-11-19',
  'epsActual': None,
  'epsEstimated': 1.23,
  'revenueActual': None,
  'revenueEstimated': 54695012750,
  'lastUpdated': '2025-11-01'},
 {'symbol': 'AAPL',
  'date': '2025-10-30',
  'epsActual': 1.85,
  'epsEstimated': 1.78,
  'revenueActual': 102466000000,
  'revenueEstimated': 102227074560,
  'lastUpdated': '2025-11-01'},
 {'symbol': 'MSFT',
  'date': '2025-10-29',
  'epsActual': 4.13,
  'epsEstimated': 3.67,
  'revenueActual': 77673000000,
  'revenueEstimated': 75494678452,
  'lastUpdated': '2025-10-31'},
 {'symbol': 'NVDA',
  'date': '2025-08-27',
  'epsActual': 1.05,
  'epsEstimated': 1.01,
  'revenueActual': 46743000000,
  'revenueEstimated': 46048920689,
  'lastUpdated': '2025-11-01'}]

In [2]:
# env: Python 3.11; requests==2.32.3; pandas==2.2.3; pyarrow==17.0.0
import requests, io, pandas as pd

FUND_CSV = {
    "ARKK": "https://assets.ark-funds.com/fund-documents/funds-etf-csv/ARK_INNOVATION_ETF_ARKK_HOLDINGS.csv",
    "ARKQ": "https://assets.ark-funds.com/fund-documents/funds-etf-csv/ARK_AUTONOMOUS_TECH._%26_ROBOTICS_ETF_ARKQ_HOLDINGS.csv",
    "ARKG": "https://assets.ark-funds.com/fund-documents/funds-etf-csv/ARK_GENOMIC_REVOLUTION_ETF_ARKG_HOLDINGS.csv",
    "ARKF": "https://assets.ark-funds.com/fund-documents/funds-etf-csv/ARK_FINTECH_INNOVATION_ETF_ARKF_HOLDINGS.csv",
    "ARKW": "https://assets.ark-funds.com/fund-documents/funds-etf-csv/ARK_NEXT_GENERATION_INTERNET_ETF_ARKW_HOLDINGS.csv",
    "ARKX": "https://assets.ark-funds.com/fund-documents/funds-etf-csv/ARK_SPACE_EXPLORATION_%26_INNOVATION_ETF_ARKX_HOLDINGS.csv",
}

def fetch_holdings_csv(url: str) -> pd.DataFrame:
    r = requests.get(url, timeout=30); r.raise_for_status()
    df = pd.read_csv(io.StringIO(r.text))
    # 简单清洗
    df.columns = [c.strip().lower().replace(" ","_") for c in df.columns]
    for c in ("shares","weight","market_value"):
        if c in df.columns:
            df[c] = (df[c].astype(str)
                     .str.replace(r"[$,%]", "", regex=True)
                     .str.replace(",", "", regex=False)
                     .replace({"": None}).astype(float))
    return df

df_arkk = fetch_holdings_csv(FUND_CSV["ARKK"])
print(df_arkk.head(3))


         date  fund                       company ticker      cusip  \
0  10/31/2025  ARKK                     TESLA INC   TSLA  88160R101   
1  10/31/2025  ARKK  COINBASE GLOBAL INC -CLASS A   COIN  19260Q107   
2  10/31/2025  ARKK                      ROKU INC   ROKU  77543R102   

      shares market_value_($) weight_(%)  
0  2263127.0  $996,002,192.70     12.30%  
1  1428669.0  $469,332,053.19      5.80%  
2  4524983.0  $452,634,049.49      5.59%  
